In [3]:
import numpy as np


items_cnt = int(input())
container_size = int(input())

items = []

for i in range(items_cnt):
    item = int(input())
    
    if item > container_size:
        print("Item weight is greater than container's capacity")
        continue
        
    items.append(item)
    
items = np.array(items)

6
1440
360
850
630
70
700
210


In [34]:
def first_fit_decr(items, container_size, is_sorted=False):
    if not is_sorted:
        sorted_inds = np.argsort(items)[::-1]
    else:
        sorted_inds = np.arange(len(items))
    
    items_location = np.zeros_like(items)
    containers_residual_size = []
    
    for item_ind in sorted_inds:
        item = items[item_ind]
        
        for cont_ind, cont_size in enumerate(containers_residual_size):
            if item < cont_size:
                items_location[item_ind] = cont_ind
                containers_residual_size[cont_ind] -= item
                break
        else:
            items_location[item_ind] = len(containers_residual_size)
            containers_residual_size.append(container_size-item)
            
    return len(containers_residual_size), items_location

first_fit_decr(items, container_size)

(2, array([0, 0, 1, 1, 1, 0]))

In [35]:
def bp_evaluation(curr_ind: int, 
                  items: np.ndarray,
                  items_inds: np.ndarray,
                  items_loc: np.ndarray, 
                  cont_sizes: np.ndarray) -> (bool, np.ndarray):
    if curr_ind == items.size:
        return True, items_loc
    
    item = items[items_inds[curr_ind]]
    
    for cont_ind, cont_size in enumerate(cont_sizes):
        if cont_size >= item:
            items_loc[items_inds[curr_ind]] = cont_ind
            cont_sizes[cont_ind] -= item
            
            is_solved, solution = bp_evaluation(curr_ind+1, items, items_inds, items_loc, cont_sizes)
            
            if is_solved:
                return is_solved, solution
            else:
                items_loc[items_inds[curr_ind]] = -1
                cont_sizes[cont_ind] += item
                
        elif curr_ind == cont_ind:
            break
            
    return False, np.array([])

def solve_bin_packing(items: np.ndarray, cap: int) -> (int, np.ndarray):
    items_loc = np.full_like(items, -1)
    sorted_inds = np.argsort(items)[::-1]
    
    lower_bound = int(np.ceil(items.sum() / cap)) - 1
    upper_bound, solution = first_fit_decr(items, cap)
    
    while upper_bound - lower_bound > 1:
        mid = (upper_bound + lower_bound) // 2
        
        is_solved, mid_solution = bp_evaluation(0, items, sorted_inds, items_loc, np.full(mid, cap))
        
        if is_solved:
            solution = mid_solution
            upper_bound = mid
        else:
            lower_bound = mid
            
    return upper_bound, solution

In [36]:
%%time
_, bp_solution = solve_bin_packing(items, container_size)
print(*bp_solution+1)

1 1 2 2 2 1
CPU times: user 1.33 ms, sys: 0 ns, total: 1.33 ms
Wall time: 875 µs


In [37]:
solve_bin_packing(np.array([10, 2, 8, 4, 7, 5]), 12)

(3, array([0, 0, 1, 1, 2, 2]))

In [117]:
%%time
oth_items = np.array([10, 15, 12, 21, 7, 9, 23, 1, 8, 4, 6, 12, 15, 20, 12, 18, 24, 31, 9, 12, 40, 18, 8, 4, 8, 12])
_, bp_solution = solve_bin_packing(oth_items, 40)
print(*bp_solution+1)

8 4 7 5 10 9 4 4 9 8 10 7 3 6 8 6 3 2 9 8 1 5 9 9 2 7
CPU times: user 32.9 s, sys: 0 ns, total: 32.9 s
Wall time: 32.9 s


In [127]:
import numpy as np


items_cnt = int(input())
container_size = int(input())

items = []

for i in range(items_cnt):
    item = int(input())
    
    if item > container_size:
        print("Item weight is greater than container's capacity")
        continue
        
    items.append(item)
    
items = np.array(items)

def first_fit_decr(items, container_size, is_sorted=False):
    if not is_sorted:
        sorted_inds = np.argsort(items)[::-1]
    else:
        sorted_inds = np.arange(len(items))
    
    items_location = np.zeros_like(items)
    containers_residual_size = []
    
    for item_ind in sorted_inds:
        item = items[item_ind]
        
        for cont_ind, cont_size in enumerate(containers_residual_size):
            if item < cont_size:
                items_location[item_ind] = cont_ind
                containers_residual_size[cont_ind] -= item
                break
        else:
            items_location[item_ind] = len(containers_residual_size)
            containers_residual_size.append(container_size-item)
            
    return len(containers_residual_size), items_location

def bp_evaluation(curr_ind: int, 
                  items: np.ndarray,
                  items_inds: np.ndarray,
                  items_loc: np.ndarray, 
                  cont_sizes: np.ndarray) -> (bool, np.ndarray):
    if curr_ind == items.size:
        return True, items_loc
    
    item = items[items_inds[curr_ind]]
    
    for cont_ind, cont_size in enumerate(cont_sizes):
        if cont_size >= item:
            items_loc[curr_ind] = cont_ind
            cont_sizes[cont_ind] -= item
            
            is_solved, solution = bp_evaluation(curr_ind+1, items, items_inds, items_loc, cont_sizes)
            
            if is_solved:
                return is_solved, solution
            else:
                items_loc[curr_ind] = -1
                cont_sizes[cont_ind] += item
                
        elif curr_ind == cont_ind:
            break
            
    return False, np.array([])

def solve_bin_packing(items: np.ndarray, cap: int) -> (int, np.ndarray):
    items_loc = np.full_like(items, -1)
    sorted_inds = np.argsort(items)[::-1]
    
    lower_bound = int(np.ceil(items.sum() / cap)) - 1
    upper_bound, solution = first_fit_decr(items, cap)
    
    while upper_bound - lower_bound > 1:
        mid = (upper_bound + lower_bound) // 2
        
        is_solved, mid_solution = bp_evaluation(0, items, sorted_inds, items_loc, np.full(mid, cap))
        
        if is_solved:
            solution = mid_solution
            upper_bound = mid
        else:
            lower_bound = mid
            
    return upper_bound, solution

_, bp_solution = solve_bin_packing(items, container_size)

dct = dict()

for x in bp_solution:
    if dct.get(x) is None:
        dct[x] = len(dct) + 1
        
    print(dct.get(x), end=' ')


KeyboardInterrupt: Interrupted by user